## 1 | Import packages

In [ ]:
import os
import h5py
import numpy as np
import tkinter
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
from nptdms import TdmsFile
print("done!")

## 2 | Use tkinter to choose a file and get the path to it

In [ ]:
# test the code
root = tkinter.Tk()
root.attributes("-topmost", True) # make window appear on top
in_path = askopenfilename()
print(in_path)
root.destroy() # close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0]
file_name = os.path.split(in_path)[1]
print(folder_name)
print(file_name)

In [ ]:
def importFile():
    "__importFile__ will open a window where you can navigate and select a file to import. It will then store its path and call the __openFile__ function to exctract its contents."
    
    root = tkinter.Tk()
    root.attributes("-topmost", True) # make window appear on top
    in_path = askopenfilename() # open dialogue to select file
    root.destroy() # close the root window

    folder_name = os.path.split(in_path)[0] # get path until folder
    file_name = os.path.split(in_path)[1] # get filename
    
    voltage_mV, current_pA, command, ttl, time, dt = openFile(in_path)
    
    return voltage_mV, current_pA, command, ttl, time, dt, folder_name

## 3 | Open a file

In [ ]:
def openFile(in_path, curated_channel = None):
    "__openFile__ will check whether you are attempting to open a tdms or a hdf5 file and will attempt to extract the data from each of the useful channels."

    if '.tdms' in in_path:
        extracted_channels, time, dt = openTDMSfile(in_path)
    elif '.hdf5' in in_path:
        extracted_channels, time, dt = openHDF5file(in_path, curated_channel = curated_channel)
    
    voltage_mV = extracted_channels[0]
    current_pA = extracted_channels[1]
    command = extracted_channels[2]
    ttl = extracted_channels[3]
    
    return voltage_mV, current_pA, command, ttl, time, dt

### 3.1 | Open an .hdf5 file and extract channels

In [ ]:
# Choose a .hdf5 file to test the code
root = tkinter.Tk()
root.attributes("-topmost", True) # make window appear on top
in_path = askopenfilename()
print(in_path)
root.destroy() # close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0]
file_name = os.path.split(in_path)[1]
print(folder_name)
print(file_name)

In [ ]:
print(in_path)
test_file = h5py.File(in_path, 'r')
list(test_file.keys())

dataDict = defaultdict(list)
list(dataDict)

In [ ]:
def openHDF5file(in_path,
                 channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B'],
                 curated_channel = None):
    "What is this function doing?"

    hdf5_file = h5py.File(in_path, 'r')
    
    dataDict = defaultdict(list)
    
    for channel in hdf5_file.keys():
        
        #to fix hdf5 indexing
        if 'Channel' in channel or 'Output' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys()) # which has been sorted alphabatically as in strings. can be simulated by sorted([str(x) for x in range(numTrials)]) 
            sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in trialKeysInHDF5]),key=lambda i:i[1])) # resort numerically, and get the sorting indexes 
            
            if curated_channel is not None: 
                sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in list(hdf5_file[curated_channel].keys())]),key=lambda i:i[1]))
                if 'Channel' in channel: trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
                if 'Output' in channel: sortI = [int(x)-1 for x in numericalOrder] #note, may not work as expected if very first trial is deleted ? but doesnt matter as output is usually the same across trials in a given file
#            print(channel, trialKeysInHDF5, sortI)
        else:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            sortI = range(len(trialKeysInHDF5))
#            print('not sorted')
            
        for i in range(len(sortI)):
            correctedTrialKey = trialKeysInHDF5[sortI[i]] 
            dataDict[channel].append(np.array(hdf5_file[channel][correctedTrialKey]))
   
    extractedChannels = []
    
    for channel in channel_list:
        extractedChannels.append(dataDict[channel])

    
    if len(dataDict['Time']) >0:
        time = dataDict['Time'][0]
        dt = np.mean(np.diff(time))
    else:
        dt = 0.04
        time = np.linspace(0, len(dataDict['Channel A'][0])*dt, len(['Channel A'][0]))
    
    return extractedChannels, time, dt

### 3.2 | Open .tdms file and extract channels

In [ ]:
# Choose a .tdms file to test the code
root = tkinter.Tk()
root.attributes("-topmost", True) # make window appear on top
in_path = askopenfilename()
print(in_path)
root.destroy() # close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0]
file_name = os.path.split(in_path)[1]
print(folder_name)
print(file_name)

In [ ]:
def openTDMSfile(in_path, channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B']):
    "__openTDMSfile__ returns a list of arrays, where each is a sweep/trial i.e. [np.array(trial)]."
    
    tdms_file = TdmsFile(in_path)
    
    dataDict = defaultdict(list)
    
    for group in tdms_file.groups():
        i=0
        for sweep in group.channels():
            dataDict[group.name].append(sweep.data)
            i+=1
                       
    extracted_channels = []
    
    for channel in channel_list: # keep only useful channels
        extracted_channels.append(dataDict[channel])
    
    time = dataDict['Time'][0]
    dt = np.mean(np.diff(time))
    
    return extracted_channels, time, dt

## 4 | Find Test Pulse and obtain seal resistance

In [ ]:
def parse_TTL_edges(TTL, edgeType):
    trial =0
    edges = []
    
    while len(edges)<1 and trial < len(TTL):
    
        if 'rising' in edgeType:
            edges = np.where(np.diff(TTL[trial])>0) 
        elif 'falling' in edgeType:
            edges = np.where(np.diff(TTL[trial])<0) 
        elif 'both' in edgeType:
            edges = np.where(np.diff(TTL[trial])!=0)    
            
        edges = edges + np.full(len(edges),1) #+1 to correct the shift due to differentiation
        edges = edges[0] #just to unwrap the array
        
        trial = trial+1
    
    
    return edges